In [1]:
import pandas as pd
import dash
from dash import Dash, dcc, html, Input, Output

In [2]:
# import data
def fetch_data():
    file_path = "companies_esg_scores.xlsx"
    df = pd.read_excel(file_path, header =0)
    return df

In [3]:
# calculate ESG Score and E Score, S Score and G Score
df = fetch_data()
total_score = df['Total_Score'].iloc[-1]
e_score = df['E_Score'].iloc[-1]
s_score = df['S_Score'].iloc[-1]
g_score = df['G_Score'].iloc[-1]

In [4]:
# Determining ESG Rating
def determine_esg_level(total_score):
    if total_score >= 85.72:
        return "AAA"
    elif total_score >= 71.44:
        return "AA"
    elif total_score >= 57.15:
        return "A"
    elif total_score >= 42.87:
        return "BBB"
    elif total_score >= 28.68:
        return "BB"
    elif total_score >= 14.30:
        return "B"
    else:
        return "CCC"

In [5]:
esg_level = determine_esg_level(total_score)

In [6]:
# generate summary and suggestions
# average score of the industry
e_avg = 20.81
s_avg = 18.89
g_avg = 12.47

function of investors

In [7]:
def generate_summary_investors(total_score, e_score, s_score, g_score):
    # Calculate esg_level
    esg_level = determine_esg_level(total_score)
    # indicators 
    e_metrics = ['Emission_intensities', 'Energy_consumption_intensity', 'Waste_generated', 'Water_intensity']
    s_metrics = ['Employee_satisfaction_rate', 'Turnover_by_gender', 'Turnover_by_age']
    g_metrics = ['Board_independence', 'Women_in_the management_team', 'Women_on_the_board']
    # average of indicators 
    e_avg_metrics = {'Emission_intensities':0.12, 'Energy_consumption_intensity':1.96, 'Waste_generated':339.90, 'Water_intensity':1.84}
    s_avg_metrics = {'Employee_satisfaction_rate':80, 'Turnover_by_gender':65.42, 'Turnover_by_age':16.28}
    g_avg_metrics = {'Board_independence': 60, 'Women_in_the management_team': 42.34, 'Women_on_the_board': 29.68}
    # Default recommendation: for companies with scores above the industry average.
    e_text = f"The score of environmental performance is {e_score.round(2)}."
    e_advice = (
    "The company's environmental performance exceeds the industry average, indicating strong sustainability practices. \n"
    "However, investors should stay attentive to changes in industry regulations and policies. \n"
    "As evolving standards may impact the company's environmental strategies and compliance requirements over time."
    )
    
    s_text = f"The score of social performance is {s_score.round(2)}."
    s_advice = (
    "The company's social performance surpasses the industry average, reflecting strong relationships with employees, customers, and communities.\n"
    "Investors are encouraged to consider this as a positive factor, \n"
    "but should continue monitoring social metrics to ensure the company maintains high standards in workforce satisfaction and other metrcis."
    )
    
    g_text = f"The score of governance performance is {g_score.round(2)}."
    g_advice = (
    "The company's governance performance is above industry average, reflecting strong management and accountability. "
    "Investors should monitor governance practices regularly to ensure they continue to align with best practices and protect shareholder interests."
    )
    
    # Different scenarios
    # Scenario1: e_score > e_avg and s_score > s_avg and g_score > g_avg
    if e_score > e_avg and s_score > s_avg and g_score > g_avg:
        e_text = f"The score of environmental performance is {e_score.round(2)}, which is higher than the average level of the healthcare industry."
        s_text = f"The score of social performance is {s_score.round(2)}, which is higher than the average level of the healthcare industry."
        g_text = f"The score of governance performance is {g_score.round(2)}, which is higher than the average level of the healthcare industry."
        
        # Step1: Identify the relatively lowest score among three scores
        lowest_score = min(e_score / 33.53, s_score / 45.90, g_score / 20.57)
        # Situation1
        if lowest_score == e_score / 33.53:           
            # find the e_metrcis that are lower than industy average
            low_e_metrics = [metric for metric in e_metrics if df[metric].iloc[-1] < e_avg_metrics[metric]]
            if low_e_metrics:
                e_advice = (
                    "Although the overall environmental score is satisfactory, the following environmental indicators are below industry average: "
                    + ", ".join(low_e_metrics) + ". "
                    "It is advisable for investors to pay attention to these areas in order to mitigate potential risks."
                )
            else:
                e_advice = (
                    "The company's environmental performance exceeds the industry average, indicating strong sustainability practices. \n"
                    "However, investors should stay attentive to changes in industry regulations and policies. \n"
                    "As evolving standards may impact the company's environmental strategies and compliance requirements over time."
                )
        
        # Situation2
        if lowest_score == s_score:
            # Identify social indicators of interest to investors that are below industry averages
            low_s_metrics = [metric for metric in s_metrics if df[metric].iloc[-1] < s_avg_metrics[metric]]
            low_s_metrics = []

            # Employee satisfaction rate
            if df['Employee_satisfaction_rate'].iloc[-1] < s_avg_metrics['Employee_satisfaction_rate']:
                low_s_metrics.append('Employee_satisfaction_rate')

            # Turnover by gender (female) and Turnover by age (aged above 50 years old)
            if df['Turnover_by_gender'].iloc[-1] > s_avg_metrics['Turnover_by_gender']:
                low_s_metrics.append('Turnover_by_gender')

            if df['Turnover_by_age'].iloc[-1] > s_avg_metrics['Turnover_by_age']:
                low_s_metrics.append('Turnover_by_age')

            if low_s_metrics:
                s_advice = (
                    "Although the overall social score is satisfactory, the following social indicators are below industry average: "
                    + ", ".join(low_s_metrics) + ". "
                    "Investors should monitor these areas for potential social impact risks."
                )
            else:
               s_advice = (
                "The company's social performance surpasses the industry average, reflecting strong relationships with employees, customers, and communities.\n"
                "Investors are encouraged to consider this as a positive factor, \n"
                "but should continue monitoring social metrics to ensure that the company maintains high standards in workforce satisfaction and community engagement over time."
                )
        
        # Situation3
        if lowest_score == g_score:
            low_g_metrics = [metric for metric in g_metrics if df[metric].iloc[-1] < g_avg_metrics[metric]]
    
            if low_g_metrics:
                g_advice = (
                    "Although the overall governance score is satisfactory, the following governance indicators are below industry average: "
                    + ", ".join(low_g_metrics) + ". "
                    "It is advisable for investors to focus on these specific governance areas to mitigate potential risks."
                )
            else:
                g_advice = (
                "The company's governance performance is above industry average, reflecting strong management and accountability. \n"
                "Investors should monitor governance practices regularly to ensure they continue to align with best practices and protect shareholder interests."
                )

    # Scenario2
    elif e_score > e_avg and s_score > s_avg and g_score <= g_avg:
        g_advice = (
            "The company demonstrates strong performance in environmental and social aspects, suggesting good sustainability practices and a positive impact on society.\n "
            "However, governance performance is below the industry average, which may pose potential risks in areas such as transparency, accountability, and decision-making.\n "
            "Investors are advised to carefully assess the company's governance structures, board diversity, and executive accountability measures to mitigate possible risks. \n"
            "Monitoring compliance with ethical standards and industry regulations is also recommended to ensure long-term stability and shareholder value."
        )
    
    elif e_score > e_avg and s_score <= s_avg and g_score > g_avg:
        s_advice = (
            "The company shows strong performance in environmental and governance aspects, indicating sound sustainability practices and effective management structures.\n"
            "However, social performance is below the industry average, which may point to challenges in areas such as employee satisfaction, workplace safety, and community relations.\n"
            "Investors are advised to assess the company's approach to social responsibility, including employee well-being, diversity, and community relations.\n"
        )
    
    elif e_score <= e_avg and s_score > s_avg and g_score > g_avg:
        e_advice = (
            "The company exhibits strong performance in social and governance aspects, suggesting effective management structures and a commitment to employee and community well-being.\n"
            "However, environmental performance is below the industry average, which may indicate potential risks related to resource management, carbon emissions, or waste management practices.\n"
            "Investors should evaluate the company's environmental policies and performance, including emissions reduction, energy efficiency, and waste management.\n"
        )

    # Scenario3
    elif e_score > e_avg and s_score <= s_avg and g_score <= g_avg:
        s_advice = (
            "The company shows strong performance in environmental aspects, indicating sound sustainability practices. \n"
            "However, both social and governance performances are below industry average, suggesting potential challenges with employee satisfaction, workplace safety, and governance accountability. \n"
            "Investors should pay more attention to the company's social responsibility initiatives to assess potential risks."
        )
        g_advice = (
            "The company's governance performance is below the industry average, suggesting possible issues with board oversight, transparency, or accountability.\n"
            "Investors are advised to closely examine the company's governance practices to evaluate risks associated with management reliability and long-term corporate stability."
        )
    
    elif e_score <= e_avg and s_score > s_avg and g_score <= g_avg:
        e_advice = (
            "The company shows strong social performance, suggesting effective workforce management and community relations. \n"
            "However, low environmental and governance scores suggest potential environmental risks and governance challenges. \n"
            "Investors are advised to investigate the company's environmental policies and governance practices to mitigate risks."
        )
        g_advice = (
            "The company's governance performance is below the industry average, which may indicate potential weaknesses in transparency, accountability, or board effectiveness.\n"
            "Investors should consider reviewing the company's governance structures and policies to assess potential risks related to management practices and decision-making processes."
        )
    
    elif e_score <= e_avg and s_score <= s_avg and g_score > g_avg:
        e_advice = (
            "The company's governance is strong, indicating sound management structures. However, environmental and social performances are below industry average, \n"
            "suggesting potential risks in sustainability and employee relations. Investors should carefully evaluate these aspects before proceeding."
        )
        s_advice = (
            "The company's social performance is below the industry average, indicating potential challenges in areas such as employee satisfaction, workplace safety, or community engagement.\n"
            "Investors may need to assess the company's efforts in improving social responsibility to reduce potential risks associated with workforce stability and public perception."
        )
        
    # Scenario4
    else:
        e_advice = (
            "The company's environmental performance is below industry standards, suggesting potential issues in areas such as emissions control, energy efficiency, and waste management.\n"
            "Investors should examine the company's policies for emissions reduction, resource optimization, and waste disposal to assess environmental risk."
        )
        s_advice = (
            "The company's social performance is also below the industry average, indicating potential issues with employee satisfaction, workplace safety, or community relations.\n"
            "Investors should investigate the company's approach to employee welfare, training, safety, and inclusion, as poor social performance can affect morale and public perception."
        )
        g_advice = (
            "Governance scores below the industry average suggest potential weaknesses in corporate transparency, board diversity, or accountability practices.\n"
            "Investors are advised to closely evaluate governance structures, including board independence, accountability, and ethical compliance, \n"
            "as poor governance may lead to misconduct and weaken shareholder value."
        )
        
    summary = (
        f"The score of ESG performance is {total_score.round(2)}, and the ranking in the industry is {esg_level}.\n\n"
        f"- {e_text}\n  {e_advice}\n\n"
        f"- {s_text}\n  {s_advice}\n\n"
        f"- {g_text}\n  {g_advice}\n"
    )
    return summary


function of regulators

In [8]:
def generate_summary_regulators(total_score, e_score, s_score, g_score):
    # Calculate esg_level
    esg_level = determine_esg_level(total_score)
    # indicators 
    e_metrics = ['Emission_intensities', 'Energy_consumption_intensity', 'Waste_generated', 'Water_intensity']
    s_metrics = ['Employee_satisfaction_rate', 'Percentage_of_employees_covered_by_health_insurance', 'Company_donated',"Work_related_injuries"]
    g_metrics = ['Board_independence', 'Women_in_the_management_team', 'Women_on_the_board']
    # average of indicators 
    e_avg_metrics = {'Emission_intensities':0.12, 'Energy_consumption_intensity':1.96, 'Waste_generated':339.90, 'Water_intensity':1.84}
    s_avg_metrics = {'Employee_satisfaction_rate':80, 'Company_donated':466528.25 ,"Work_related_injuries": 4.7}
    g_avg_metrics = {'Board_independence': 60, 'Women_in_the_management_team': 42.34, 'Women_on_the_board': 29.68}
    # Default recommendation: for companies with scores above the industry average.
    e_text = f"The score of environmental performance is {e_score.round(2)}."
    e_advice = (
        "The company's environmental performance exceeds the industry average, indicating strong sustainability practices.\n"
        "However, regulatory agencies should monitor changes in environmental policies and standards to ensure the company's practices continue to align with evolving requirements.\n"
        "Attention should be given to the company's ability to adapt its environmental strategies to comply with regulatory updates."
    )
    
    s_text = f"The score of social performance is {s_score.round(2)}."
    s_advice = (
        "The company's social performance surpasses the industry average, reflecting strong relationships with employees, customers, and communities.\n"
        "Regulatory agencies should encourage the company to maintain these standards, while also monitoring any fluctuations in workforce satisfaction and other social metrics."
    )
    
    g_text = f"The score of governance performance is {g_score.round(2)}."
    g_advice = (
        "The company's governance performance is above the industry average, reflecting strong management and accountability.\n"
        "Regulatory bodies should continue to ensure that governance practices align with industry standards, focusing on transparency, accountability, and compliance with ethical guidelines."
    )
    
    # Different scenarios
    # Scenario1: e_score > e_avg and s_score > s_avg and g_score > g_avg
    if e_score > e_avg and s_score > s_avg and g_score > g_avg:
        e_text = f"The score of environmental performance is {e_score.round(2)}, which is higher than the average level of the healthcare industry."
        s_text = f"The score of social performance is {s_score.round(2)}, which is higher than the average level of the healthcare industry."
        g_text = f"The score of governance performance is {g_score.round(2)}, which is higher than the average level of the healthcare industry."
        
        # Step1: Identify the relatively lowest score among three scores
        lowest_score = min(e_score / 33.53, s_score / 45.90, g_score / 20.57)
        
        # Situation1
        if lowest_score == e_score / 33.53:           
            low_e_metrics = [metric for metric in e_metrics if df[metric].iloc[-1] < e_avg_metrics[metric]]
            if low_e_metrics:
                e_advice = (
                    "Although the company's overall environmental performance is strong, the following environmental indicators are below industry average: "
                    + ", ".join(low_e_metrics) + ".\n"
                    "Regulatory agencies should pay close attention to these areas to encourage improvement and ensure the company remains compliant with environmental standards."
                )
            else:
                e_advice = (
                    "The company's environmental performance exceeds the industry average, indicating strong sustainability practices.\n"
                    "However, regulatory agencies should monitor changes in environmental policies to ensure continued alignment with updated standards."
                )
        
        # Situation2
        if lowest_score == s_score / 45.90:
            low_s_metrics = []
            
            if df['Employee_satisfaction_rate'].iloc[-1] < s_avg_metrics['Employee_satisfaction_rate']:
                low_s_metrics.append('Employee_satisfaction_rate')

            if df['Company_donated'].iloc[-1] < s_avg_metrics['Company_donated']:
                low_s_metrics.append('Company_donated')

            if df["Work_related_injuries"].iloc[-1] > s_avg_metrics["Work_related_injuries"]:
                low_s_metrics.append("Work_related_injuries")

            if low_s_metrics:
                s_advice = (
                    "Although the company's social performance is generally strong, the following social indicators are below industry average: "
                    + ", ".join(low_s_metrics) + ".\n"
                    "Regulatory agencies should monitor these areas to ensure the company addresses any underlying issues and promotes a healthy social environment."
                )
            else:
                s_advice = (
                    "The company's social performance surpasses the industry average, reflecting strong relationships with employees, customers, and communities.\n"
                    "Regulatory agencies should encourage the company to maintain these standards, while monitoring any significant changes."
                )
        
        # Situation3
        if lowest_score == g_score / 20.57:
            low_g_metrics = [metric for metric in g_metrics if df[metric].iloc[-1] < g_avg_metrics[metric]]
    
            if low_g_metrics:
                g_advice = (
                    "Although the company's governance score is generally high, the following governance indicators are below industry average: "
                    + ", ".join(low_g_metrics) + ".\n"
                    "Regulatory agencies should focus on these governance aspects to promote accountability and prevent potential risks."
                )
            else:
                g_advice = (
                    "The company's governance performance is above industry average, reflecting strong management and accountability.\n"
                    "Regulatory agencies should ensure these practices align with industry standards for sustainable growth and compliance."
                )

    # Other Scenarios
    elif e_score > e_avg and s_score > s_avg and g_score <= g_avg:
        g_advice = (
            "The company demonstrates strong performance in environmental and social aspects, which reflects positively on its sustainability practices.\n"
            "However, governance performance is below industry average. Regulatory agencies should review the company's governance structures to improve transparency, accountability, and decision-making processes."
        )
    
    elif e_score > e_avg and s_score <= s_avg and g_score > g_avg:
        s_advice = (
            "The company shows strength in environmental and governance aspects, but social performance is below industry average.\n"
            "Regulatory agencies should assess the company's social responsibility initiatives, particularly around workforce satisfaction, to encourage improvement."
        )
    
    elif e_score <= e_avg and s_score > s_avg and g_score > g_avg:
        e_advice = (
            "The company performs well in social and governance aspects, but its environmental performance is below industry average.\n"
            "Regulatory agencies should focus on the company's environmental practices to ensure alignment with industry standards and encourage sustainable improvements."
        )

    elif e_score > e_avg and s_score <= s_avg and g_score <= g_avg:
        s_advice = (
            "The company has strong environmental practices, but social and governance performances are below industry average.\n"
            "Regulatory agencies should emphasize improvements in social responsibility and governance accountability."
        )
        g_advice = (
            "The company's governance is below industry average, indicating potential issues with oversight and transparency.\n"
            "Regulatory agencies should examine governance practices to mitigate management and compliance risks."
        )
    
    elif e_score <= e_avg and s_score > s_avg and g_score <= g_avg:
        e_advice = (
            "The company demonstrates good social performance, but its environmental and governance scores are below average.\n"
            "Regulatory agencies should prioritize improvements in environmental practices and governance mechanisms."
        )
        g_advice = (
            "With governance performance below the industry average, regulatory agencies should assess the company’s governance structures to address potential accountability issues."
        )
    
    elif e_score <= e_avg and s_score <= s_avg and g_score > g_avg:
        e_advice = (
            "The company's governance is strong, but its environmental and social performances are below industry average.\n"
            "Regulatory agencies should consider interventions to improve the company's sustainability and social responsibility efforts."
        )
        s_advice = (
            "The company's social performance falls short of industry standards, indicating areas for improvement in employee and community relations.\n"
            "Regulatory agencies should encourage efforts in these areas to improve social impact."
        )
        
    # All Scores Below Industry Average
    else:
        e_advice = (
            "The company's environmental performance is below industry standards, suggesting possible issues with emissions, energy efficiency, and waste management.\n"
            "Regulatory agencies should work with the company to address these issues and improve its environmental impact."
        )
        s_advice = (
            "The company's social performance is also below industry average, which may indicate issues with employee satisfaction and community engagement.\n"
            "Regulatory agencies should encourage improvements in social responsibility and workplace conditions."
        )
        g_advice = (
            "The company's governance is below industry standards, highlighting potential weaknesses in transparency, accountability, and ethical compliance.\n"
            "Regulatory agencies should review and promote better governance practices to safeguard shareholder interests."
        )
        
    # Generate summary and suggestions
    summary = (
        f"The score of ESG performance is {total_score.round(2)}, and the ranking in the industry is {esg_level}.\n\n"
        f"- {e_text}\n  {e_advice}\n\n"
        f"- {s_text}\n  {s_advice}\n\n"
        f"- {g_text}\n  {g_advice}\n"
    )
    return summary


function of company management

In [9]:
def generate_summary_management(total_score, e_score, s_score, g_score):
    # Calculate esg_level
    esg_level = determine_esg_level(total_score)
    # indicators
    e_metrics = ['Emission_intensities', 'Energy_consumption_intensity', 'Waste_generated', 'Water_intensity']
    s_metrics = ['Employee_satisfaction_rate', 'Percentage_of_employees_covered_by_health_insurance', 'Company_donated',"Work_related_injuries"]
    g_metrics = ['Board_independence', 'Women_in_the_management_team', 'Women_on_the_board']
    # average of indicators 
    e_avg_metrics = {'Emission_intensities':0.12, 'Energy_consumption_intensity':1.96, 'Waste_generated':339.90, 'Water_intensity':1.84}
    s_avg_metrics = {'Employee_satisfaction_rate':80, 'Company_donated':466528.25 ,"Work_related_injuries": 4.7}
    g_avg_metrics = {'Board_independence': 60, 'Women_in_the_management_team': 42.34, 'Women_on_the_board': 29.68}
    
    # Default recommendation: for companies with scores above the industry average.
    e_text = f"The score of environmental performance is {e_score.round(2)}."
    e_advice = (
        "The company's environmental performance exceeds the industry average, demonstrating strong sustainability practices.\n"
        "Management should continue to monitor environmental regulations and industry standards to ensure compliance and maintain leadership in environmental responsibility.\n"
        "It's essential to adapt environmental strategies proactively to align with potential regulatory changes."
    )
    
    s_text = f"The score of social performance is {s_score.round(2)}."
    s_advice = (
        "The company's social performance surpasses the industry average, reflecting positive relations with employees, customers, and communities.\n"
        "Management should focus on maintaining these standards and consider enhancing social initiatives to further strengthen workforce satisfaction and community engagement."
    )
    
    g_text = f"The score of governance performance is {g_score.round(2)}."
    g_advice = (
        "The company's governance performance is above industry average, indicating strong management and accountability structures.\n"
        "Management should regularly review governance policies to ensure they align with best practices and reinforce transparency and ethical compliance across all levels."
    )
    
    # Different scenarios
    # Scenario1: e_score > e_avg and s_score > s_avg and g_score > g_avg
    if e_score > e_avg and s_score > s_avg and g_score > g_avg:
        e_text = f"The score of environmental performance is {e_score.round(2)}, which is higher than the average level of the healthcare industry."
        s_text = f"The score of social performance is {s_score.round(2)}, which is higher than the average level of the healthcare industry."
        g_text = f"The score of governance performance is {g_score.round(2)}, which is higher than the average level of the healthcare industry."
        
        # Step1: Identify the relatively lowest score among three scores
        lowest_score = min(e_score / 33.53, s_score / 45.90, g_score / 20.57)
        
        # Situation1
        if lowest_score == e_score / 33.53:           
            low_e_metrics = [metric for metric in e_metrics if df[metric].iloc[-1] < e_avg_metrics[metric]]
            if low_e_metrics:
                e_advice = (
                    "Although the company's environmental performance is strong overall, the following environmental metrics are below the industry average: "
                    + ", ".join(low_e_metrics) + ".\n"
                    "Management should focus on improving these specific areas to ensure comprehensive environmental leadership."
                )
            else:
                e_advice = (
                    "The company's environmental performance exceeds the industry average. Management should continue to monitor environmental policies to ensure ongoing compliance and proactive alignment with regulatory standards."
                )
        
        # Situation2
        if lowest_score == s_score / 45.90:
            low_s_metrics = []
            
            if df['Employee_satisfaction_rate'].iloc[-1] < s_avg_metrics['Employee_satisfaction_rate']:
                low_s_metrics.append('Employee_satisfaction_rate')

            if df['Company_donated'].iloc[-1] < s_avg_metrics['Company_donated']:
                low_s_metrics.append('Company_donated')

            if df["Work_related_injuries"].iloc[-1] > s_avg_metrics["Work_related_injuries"]:
                low_s_metrics.append("Work_related_injuries")

            if low_s_metrics:
                s_advice = (
                    "While the company's social performance is strong overall, the following social metrics are below industry average: "
                    + ", ".join(low_s_metrics) + ".\n"
                    "Management should address these areas to promote a healthier work environment and stronger community relations."
                )
            else:
                s_advice = (
                    "The company's social performance surpasses the industry average. Management should maintain these standards while proactively monitoring any shifts in employee or community relations."
                )
        
        # Situation3
        if lowest_score == g_score / 20.57:
            low_g_metrics = [metric for metric in g_metrics if df[metric].iloc[-1] < g_avg_metrics[metric]]
    
            if low_g_metrics:
                g_advice = (
                    "Although the company's governance is strong overall, the following governance metrics are below industry average: "
                    + ", ".join(low_g_metrics) + ".\n"
                    "Management should focus on improving these areas to strengthen transparency and accountability."
                )
            else:
                g_advice = (
                    "The company's governance performance is above industry average. Management should continue to align governance practices with industry best practices to maintain a robust accountability structure."
                )

    # Other Scenarios
    elif e_score > e_avg and s_score > s_avg and g_score <= g_avg:
        g_advice = (
            "The company shows strong environmental and social performance, indicating solid sustainability practices.\n"
            "However, governance performance is below industry average. Management should focus on enhancing governance structures to improve transparency and decision-making processes."
        )
    
    elif e_score > e_avg and s_score <= s_avg and g_score > g_avg:
        s_advice = (
            "The company demonstrates strength in environmental and governance aspects, but social performance is below industry average.\n"
            "Management should consider bolstering social responsibility efforts, particularly focusing on workforce satisfaction and community engagement."
        )
    
    elif e_score <= e_avg and s_score > s_avg and g_score > g_avg:
        e_advice = (
            "The company performs well in social and governance aspects, but environmental performance is below industry average.\n"
            "Management should review environmental policies and practices to align with industry standards and drive improvement."
        )

    elif e_score > e_avg and s_score <= s_avg and g_score <= g_avg:
        s_advice = (
            "The company shows strong environmental practices, but social and governance performances are below industry average.\n"
            "Management should prioritize improvements in social responsibility and governance accountability to mitigate potential risks."
        )
        g_advice = (
            "Governance performance is below industry average, suggesting potential gaps in oversight and transparency.\n"
            "Management should reinforce governance practices to improve accountability and strengthen corporate integrity."
        )
    
    elif e_score <= e_avg and s_score > s_avg and g_score <= g_avg:
        e_advice = (
            "The company demonstrates good social performance, but its environmental and governance scores are below average.\n"
            "Management should enhance environmental policies and governance structures to support long-term resilience and regulatory alignment."
        )
        g_advice = (
            "With governance performance below industry average, management should review and strengthen governance structures to address accountability issues."
        )
    
    elif e_score <= e_avg and s_score <= s_avg and g_score > g_avg:
        e_advice = (
            "The company's governance is strong, but its environmental and social performances are below industry average.\n"
            "Management should invest in improving sustainability practices and social responsibility to better meet industry standards."
        )
        s_advice = (
            "The company's social performance falls short of industry standards, indicating potential challenges in workforce and community relations.\n"
            "Management should address these areas to enhance social impact and company reputation."
        )
        
    # All Scores Below Industry Average
    else:
        e_advice = (
            "The company's environmental performance is below industry standards, suggesting potential issues in emissions control, energy efficiency, and waste management.\n"
            "Management should focus on implementing sustainable practices to improve environmental impact and compliance."
        )
        s_advice = (
            "The company's social performance is also below industry average, indicating possible issues with employee satisfaction and community engagement.\n"
            "Management should work to improve social initiatives and strengthen relationships with both employees and the broader community."
        )
        g_advice = (
            "Governance performance is below industry standards, highlighting potential weaknesses in transparency and accountability.\n"
            "Management should strengthen governance practices to improve oversight, accountability, and alignment with ethical standards."
        )
        
    # Generate summary and suggestions
    summary = (
        f"The score of ESG performance is {total_score.round(2)}, and the ranking in the industry is {esg_level}.\n\n"
        f"- {e_text}\n  {e_advice}\n\n"
        f"- {s_text}\n  {s_advice}\n\n"
        f"- {g_text}\n  {g_advice}\n"
    )
    return summary


In [10]:
app = Dash(__name__)

In [11]:
management_summary = generate_summary_management(total_score, e_score, s_score, g_score)
regulator_summary = generate_summary_regulators(total_score, e_score, s_score, g_score)
investor_summary = generate_summary_investors(total_score, e_score, s_score, g_score)

In [12]:
app.layout = html.Div([
    html.H1("ESG Report Summaries and Recommendations", style={'textAlign': 'left', 'marginBottom': '20px'}),
    
    html.H2("Recommendations for Management", style={'textAlign': 'left'}),
    dcc.Textarea(
        id='management-summary',
        value=management_summary,
        style={
            'width': '70%',
            'height': '300px',
            'margin': 'auto',
            'padding': '10px',
            'fontFamily': 'Arial',
            'fontSize': '16px',
            'backgroundColor': '#ffffff',
            'color': '#333333',
            'lineHeight': '1.6',
            'textAlign': 'left',
            'whiteSpace': 'pre-wrap',
            'whiteSpace': 'pre-line'
        },
        readOnly=True
    ),
    
    html.H2("Recommendations for Regulators", style={'textAlign': 'left', 'marginTop': '30px'}),
    dcc.Textarea(
        id='regulator-summary',
        value=regulator_summary,
        style={
            'width': '70%',
            'height': '300px',
            'margin': 'auto',
            'padding': '10px',
            'fontFamily': 'Arial',
            'fontSize': '16px',
            'backgroundColor': '#ffffff',
            'color': '#333333',
            'lineHeight': '1.6',
            'textAlign': 'left',
            'whiteSpace': 'pre-wrap',
            'whiteSpace': 'pre-line'
        },
        readOnly=True
    ),
    
    html.H2("Recommendations for Investors", style={'textAlign': 'left', 'marginTop': '30px'}),
    dcc.Textarea(
        id='investor-summary',
        value=investor_summary,
        style={
            'width': '70%',
            'height': '300px',
            'margin': 'auto',
            'padding': '10px',
            'fontFamily': 'Arial',
            'fontSize': '16px',
            'backgroundColor': '#ffffff',
            'color': '#333333',
            'lineHeight': '1.6',
            'textAlign': 'left',
            'whiteSpace': 'pre-wrap',
            'whiteSpace': 'pre-line'
        },
        readOnly=True
    )
])

In [13]:
if __name__ == '__main__':
    app.run_server(debug=True, port=8091)